In [ ]:
import tensorflow as tf
from tensorflow import keras

# Common imports
import numpy as np
import os
from functools import partial
from matplotlib import pyplot as plt
%pylab inline
import warnings
warnings.filterwarnings('ignore')
from keras.utils import np_utils

np.random.seed(42)

# stuff to make this work on GPUs
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [ ]:
!pip install emnist
from emnist import extract_training_samples
from emnist import extract_test_samples
X_train, y_train = extract_training_samples('letters')
X_test, y_test = extract_test_samples('letters')
X_train = X_train/255.
X_test = X_test/255.
# jedes Pixel in allen Bildern einen Wert zwischen 0 und 1 hat
print(X_train.shape)
print(X_test.shape)
# 124800 Bilder fürs Trainieren
# 20800 Bilder fürs Testen

In [ ]:
img_index_train = 8888 # das img_index. Image ausnehmen
img_train = X_train[img_index_train]
img_index_test = 1888
img_test = X_test[img_index_test]
plt.figure(figsize=(3,8))
plt.subplot(2, 1, 1)
plt.imshow(img_train.reshape((28,28)), cmap='gray')
plt.title(chr(y_train[img_index_train]+96))
plt.subplot(2, 1, 2)
plt.imshow(img_test.reshape((28,28)))
plt.title(chr(y_test[img_index_test]+96))

In [ ]:
n_train = X_train.shape[0]
n_test = X_test.shape[0]
image_shape = X_train[0].shape
n_classes = len(np.unique(y_train))
random.seed(4242)

print("Number of Training Images =", n_train)
print("Number of Test Images =    ", n_test)
print("Image Format =             ", image_shape)
print("Number of Classes =        ", n_classes)

In [ ]:
y_train_cat = np_utils.to_categorical(y_train-1, 26)
y_test_cat = np_utils.to_categorical(y_test-1, 26)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print("Shape before = ", X_train.shape)
X_train_reshaped = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test_reshaped = X_test.reshape(X_test.shape[0], 28, 28, 1)
print("Shape after  = ", X_train_reshaped.shape)

In [ ]:
BATCH_SIZE = 100
EPOCHS     = 20

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=3, strides=1, input_shape=(28,28,1)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=2, strides=2, padding="SAME"))
model.add(keras.layers.Dropout(0.5))
prev_filters = 64

for filters in [64] * 2 + [128] * 2 + [256] * 2:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(n_classes, activation="softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train_reshaped, y_train_cat, 
          batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1)

In [ ]:
score = model.evaluate(X_test_reshaped, y_test_cat)

print('Result on Test Set')
print('Test loss:    ', score[0])
print('Test accuracy:', score[1])

In [ ]:
resultingClasses = np.argmax(model.predict(X_test_reshaped), axis=1)
incorrects = np.nonzero(resultingClasses != y_test-1)
print("Total number of misclassifications: ", len(incorrects[0]))

In [ ]:
plt.figure(figsize=(18,22))
index = 0
for imageNumber in incorrects[0]:
    #print(imageNumber)
    plt.subplot(20,20,index+1)
    plt.imshow(X_test[imageNumber], cmap='gray')
    plt.title("{} as {}".format(chr(y_test[imageNumber]+96), chr(resultingClasses[imageNumber]+96)))
    plt.xticks([])
    plt.yticks([])
    index = index + 1
    if index >= 400:
        break